In [4]:
import torch 
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo" 

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
    chunk_length_s=10,
    stride_length_s=2,
)

sanoke = "audio.mp3"

result = pipe(sanoke)

print(result)

Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


{'text': ' 여러분이 승진하길 바란다고 가정해 본다면 누군가에게 축하받는 일은 승진 후에 뒤따라올만한 일. 승진했다면 뒤따라 올만한 사건으로 축하받는 장면을 골랐다면 몸을 움직이지 말고 잠과의 경계상태인 졸린 상태를 만듭니다. 그러나 졸린 상태 속에서도 여전히 생각을 조절할 수 있어야 합니다. 애쓰지 않고 의식을 기울일 수 있는 상태가 되어야 합니다. 그런 후에 친구가 옆에 서 있는 것을 그립니다. 이제는 여러분의 상상 속의 손을 친구의 손과 잡습니다. 그 손을 단단한 실체처럼 느끼십시오. 그리고 승진했다는 느낌과 어울릴만한 상상 속의 대화를 나눕니다. 여러분의 소망은 지금 이곳에서 이루어졌습니다. 그래서 그 시간은 미래에 저 멀리 떨어진 어떤 때라 생각하면 안 됩니다. 여러분이 승진을 해서 축하를 받는 그 시간은 바로 지금이고 만약 그 장소가 사무실이라면 이곳이 바로 사무실인 겁니다. 그 장소를 이곳으로 만들고 그 시간을 미래가 아닌 지금 현재로 만들어야 합니다. 다시 말해 그 사건을 지금 여기서 겪는다고 생각해야 합니다. 여러분이 지금 여기에서 행동하는 것을 느끼는 것과 영화 스크린에 그 행동을 그리는 것의 차이는 성공과 실패를 결정짓는 중요한 차이입니다. 사다리에 오르는 것을 지금 그려본다면 그 차이를 알 수 있을 겁니다. 눈을 감은 후 사다리가 눈앞에 있는 것을 그린 후에 그것을 실제 올라가는 것을 느껴보십시오. 제 경험상 마음 안에서 소망이 성취된 것을 나타내는 행동을 할 때 그 행동을 특정한 범위 안으로 제한시켜 간단한 행동으로 압축시킨 후에 그 이미지가 현실처럼 느껴질 때까지 계속 반복하는 것이 좋습니다. 그렇게 하지 않는다면 여러분의 주의력은 연상되는 것들을 따라서 이리저리 돌아다니다가 수많은 연상되는 다른 이미지들을 따라 가게 됩니다. 결국은 여러분의 의식은 처음에 계획했던 이미지의 장소보다 훨씬 더 멀리로 끌려가 있고 처음에 계획했던 시간보다 몇 년은 더 멀리 떨어진 시간에 머물게 됩니다. 계단을 올라가는 것이 여러분의 바람이 성취되었

In [6]:
start_end_text = []
text_list = []

for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append(f"{start:.2f} - {end:.2f}: {text}")
    text_list.append(text)

text_list = " ".join(text_list)

print("\n".join(start_end_text))


0.00 - 8.06:  여러분이 승진하길 바란다고 가정해 본다면 누군가에게 축하받는 일은 승진 후에 뒤따라올만한 일. 승진했다면
8.06 - 10.12:  뒤따라 올만한 사건으로 축하받는
10.12 - 11.12:  장면을 골랐다면
11.12 - 13.70:  몸을 움직이지 말고 잠과의
13.70 - 15.80:  경계상태인 졸린 상태를 만듭니다.
16.46 - 19.98:  그러나 졸린 상태 속에서도 여전히 생각을 조절할 수 있어야 합니다.
20.70 - 21.90:  애쓰지 않고 의식을
21.90 - 23.78:  기울일 수 있는 상태가 되어야 합니다.
24.48 - 25.86:  그런 후에 친구가 옆에
25.86 - 26.92:  서 있는 것을 그립니다.
27.68 - 30.80:  이제는 여러분의 상상 속의 손을 친구의 손과 잡습니다.
31.54 - 33.82:  그 손을 단단한 실체처럼 느끼십시오.
34.42 - 35.94:  그리고 승진했다는 느낌과
35.94 - 38.16:  어울릴만한 상상 속의 대화를 나눕니다.
38.88 - 41.48:  여러분의 소망은 지금 이곳에서 이루어졌습니다.
42.20 - 46.10:  그래서 그 시간은 미래에 저 멀리 떨어진 어떤 때라 생각하면 안 됩니다.
46.90 - 50.20:  여러분이 승진을 해서 축하를 받는 그 시간은 바로 지금이고
50.20 - 51.88:  만약 그 장소가
51.88 - 53.82:  사무실이라면 이곳이 바로
53.82 - 56.06:  사무실인 겁니다. 그 장소를
56.06 - 57.96:  이곳으로 만들고 그 시간을 미래가
57.96 - 59.76:  아닌 지금 현재로 만들어야 합니다.
60.46 - 64.32:  다시 말해 그 사건을 지금 여기서 겪는다고 생각해야 합니다.
65.06 - 74.48:  여러분이 지금 여기에서 행동하는 것을 느끼는 것과 영화 스크린에 그 행동을 그리는 것의 차이는 성공과 실패를 결정짓는 중요한 차이입니다.
75.16 - 76.24:  사다리에 오르는

In [9]:
print(text_list)

 여러분이 승진하길 바란다고 가정해 본다면 누군가에게 축하받는 일은 승진 후에 뒤따라올만한 일. 승진했다면  뒤따라 올만한 사건으로 축하받는  장면을 골랐다면  몸을 움직이지 말고 잠과의  경계상태인 졸린 상태를 만듭니다.  그러나 졸린 상태 속에서도 여전히 생각을 조절할 수 있어야 합니다.  애쓰지 않고 의식을  기울일 수 있는 상태가 되어야 합니다.  그런 후에 친구가 옆에  서 있는 것을 그립니다.  이제는 여러분의 상상 속의 손을 친구의 손과 잡습니다.  그 손을 단단한 실체처럼 느끼십시오.  그리고 승진했다는 느낌과  어울릴만한 상상 속의 대화를 나눕니다.  여러분의 소망은 지금 이곳에서 이루어졌습니다.  그래서 그 시간은 미래에 저 멀리 떨어진 어떤 때라 생각하면 안 됩니다.  여러분이 승진을 해서 축하를 받는 그 시간은 바로 지금이고  만약 그 장소가  사무실이라면 이곳이 바로  사무실인 겁니다. 그 장소를  이곳으로 만들고 그 시간을 미래가  아닌 지금 현재로 만들어야 합니다.  다시 말해 그 사건을 지금 여기서 겪는다고 생각해야 합니다.  여러분이 지금 여기에서 행동하는 것을 느끼는 것과 영화 스크린에 그 행동을 그리는 것의 차이는 성공과 실패를 결정짓는 중요한 차이입니다.  사다리에 오르는 것을  지금 그려본다면 그 차이를  알 수 있을 겁니다.  눈을 감은 후 사다리가 눈앞에 있는 것을  그린 후에 그것을 실제 올라가는 것을  느껴보십시오.  제 경험상 마음 안에서 소망이 성취된 것을 나타내는 행동을 할 때  그 행동을 특정한 범위 안으로 제한시켜 간단한 행동으로  압축시킨 후에 그 이미지가 현실처럼 느껴질 때까지 계속 반복하는 것이 좋습니다.  그렇게 하지 않는다면  여러분의 주의력은 연상되는 것들을 따라서 이리저리 돌아다니다가 수많은  연상되는 다른 이미지들을 따라 가게 됩니다.  결국은 여러분의 의식은 처음에 계획했던 이미지의 장소보다 훨씬 더 멀리로 끌려가 있고 처음에 계획했던  시간보다 몇 년은 더 멀리  떨어진 시간에 머물게 됩니